## Singleton Demo

The `Status` class below is implemented as a `Singleton`. Imagine that its `_status_update` method is resource heavy and we want to limit calls to it. Having a singleton makes sure only one instance exists which makes the calls. Caching the status further reduces costly status updates.

Note that the two `Caller` objects update more frequently. They both share the same `Status` singleton and you can see the status caching in action.

In [1]:
import time
import datetime as dt


class Singleton(type):
    """Implements singeton as metaclass."""

    # Keep track of instances
    _instances = {}
    
    def __call__(cls, *args, **kwargs):
        """Class construction, return exiting instance or create one."""
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

class Status(metaclass=Singleton):
    """Status class updates status at regular intervals."""
    
    def __init__(self, freq=5):
        """Constructor, set update frequency in seconds."""
        self._freq = freq
        
        # Set initial status and update time
        self._status = 0
        self._last_update = dt.datetime.now()
        
    def _update_status(self):
        """Perform status update if status has expired."""
        delta = dt.datetime.now() - self._last_update
        if delta.total_seconds() > self._freq:
            self._status += 1
            self._last_update = dt.datetime.now()

    def get_status(self):
        """Returns most recent status."""
        self._update_status()
        return self._status
    
class Caller:
    """Caller class that uses above Status class."""
    
    def __init__(self):
        self._status = Status()
    
    def update(self):
        return self._status.get_status()

In [2]:
# Create 2 callers
caller1 = Caller()
caller2 = Caller()

# Note how both callers have the same Status object.
# Having one Status object limits overall _status_update calls.
print(f"caller1._status id: {id(caller1._status)}")
print(f"caller2._status id: {id(caller2._status)}")

caller1._status id: 1449813596360
caller2._status id: 1449813596360


In [3]:
# Loop every second, watch the status change every 5 seconds
# Note: both callers get the same status!
for loop in range(1, 15):
    print(f"Loop {loop}: caller1.update() returns: {caller1.update()}")
    print(f"Loop {loop}: caller2.update() returns: {caller2.update()}")
    time.sleep(1)

Loop 1: caller1.update() returns: 0
Loop 1: caller2.update() returns: 0
Loop 2: caller1.update() returns: 0
Loop 2: caller2.update() returns: 0
Loop 3: caller1.update() returns: 0
Loop 3: caller2.update() returns: 0
Loop 4: caller1.update() returns: 0
Loop 4: caller2.update() returns: 0
Loop 5: caller1.update() returns: 0
Loop 5: caller2.update() returns: 0
Loop 6: caller1.update() returns: 1
Loop 6: caller2.update() returns: 1
Loop 7: caller1.update() returns: 1
Loop 7: caller2.update() returns: 1
Loop 8: caller1.update() returns: 1
Loop 8: caller2.update() returns: 1
Loop 9: caller1.update() returns: 1
Loop 9: caller2.update() returns: 1
Loop 10: caller1.update() returns: 1
Loop 10: caller2.update() returns: 1
Loop 11: caller1.update() returns: 2
Loop 11: caller2.update() returns: 2
Loop 12: caller1.update() returns: 2
Loop 12: caller2.update() returns: 2
Loop 13: caller1.update() returns: 2
Loop 13: caller2.update() returns: 2
Loop 14: caller1.update() returns: 2
Loop 14: caller2.up